In [1]:
import pandas as pd
import numpy as np
import datetime
import sys, os
import glob
import time
import fnmatch
import os
import xlwings as xw
from datetime import date
start_time = time.time()

print('Reading MRP file')

a=fnmatch.filter(os.listdir('.'), 'MRP_*')
a=a[0]

dfMRP = pd.read_excel(a,dtype={'ITEM': str, 'ITEM_PLAN': str}, sheet_name='MRP FINISHED')

dfMRP.head()

#delete column with no header if exists
if set(['Unnamed: 97']).issubset(dfMRP.columns):
   dfMRP=dfMRP.drop(['Unnamed: 97'], axis = 1)

dfMRPjustdates= dfMRP.drop(['ORG','ITEM','ITEM DESCRIPTION','ITEM STATUS','USER ITEM TYPE','DEFAULT BUYER',
                             'MAKE OR BUY','PLANNING METHOD','ITEM ORGANIZATION SOURCING RULE','ZEBRA ABCD RATING',
                             'ZEBRA PURCHASING TYPE','ZEBRA PLANNING CATEGORY','FAMILY','PO NUMBER','PO  QTY DUE',
                             'LEAD TIME','PO UNIT PRICE','OH','Grand Total','SRDC DEMAND SUPPLY QTY','TRUE OR FALSE'], axis = 1)
#dfMRPjustdates.head()

dfMRPjustdates = dfMRPjustdates.set_index('ITEM_PLAN')
dfMRPjustdates.columns = pd.to_datetime(dfMRPjustdates.columns)
dfMRPjustdates

#get date of june 30 2022
june30 = datetime.datetime(2022, 6, 30)
#subset of rows before june 30
dfMRPbeforejune30 = dfMRPjustdates.loc[:, dfMRPjustdates.columns <= june30]
dfMRPbeforejune30['BeforeJune30']=dfMRPbeforejune30.iloc[:].sum(axis=1) #df['Sum']=df.iloc[:,[1,2]].sum(axis=1)
#dfMRPbeforejune30.head()
#subset of rows after june 30
dfMRPafterjune30 = dfMRPjustdates.loc[:, dfMRPjustdates.columns > june30]
dfMRPafterjune30['AfterJune30']=dfMRPafterjune30.iloc[:].sum(axis=1) #df['Sum']=df.iloc[:,[1,2]].sum(axis=1)
#dfMRPafterjune30.head()

dfMRPafterjune30.drop(dfMRPafterjune30.columns.difference(['AfterJune30']), 1, inplace=True)
dfMRPbeforejune30.drop(dfMRPbeforejune30.columns.difference(['BeforeJune30']), 1, inplace=True)
dfMRPafterjune30.head()

dfMRP= pd.merge(dfMRP,dfMRPbeforejune30, on='ITEM_PLAN', how='left')
dfMRP= pd.merge(dfMRP,dfMRPafterjune30, on='ITEM_PLAN', how='left')
dfMRP.head()

Reading MRP file


<ipython-input-1-334aa28f1d47>:40: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfMRPbeforejune30['BeforeJune30']=dfMRPbeforejune30.iloc[:].sum(axis=1) #df['Sum']=df.iloc[:,[1,2]].sum(axis=1)
<ipython-input-1-334aa28f1d47>:44: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfMRPafterjune30['AfterJune30']=dfMRPafterjune30.iloc[:].sum(axis=1) #df['Sum']=df.iloc[:,[1,2]].sum(axis=1)
C:\Users\MJ7255\Anaconda3\lib\site-packages\pandas\core\frame.py:4163: SettingWithCopyWarning: 
A value is trying to be set on a 

,ORG,ITEM,ITEM_PLAN,ITEM DESCRIPTION,ITEM STATUS,USER ITEM TYPE,DEFAULT BUYER,MAKE OR BUY,PLANNING METHOD,ITEM ORGANIZATION SOURCING RULE,...,2023-10-23 00:00:00,2023-10-30 00:00:00,2023-11-06 00:00:00,2023-11-13 00:00:00,2023-11-20 00:00:00,Grand Total,SRDC DEMAND SUPPLY QTY,TRUE OR FALSE,BeforeJune30,AfterJune30
0,NL1,00358-000,00358-000NL1,CABLE CLAMP,FG-Active,FG Purchased Parts,"Mak, Ms. Ee Tinh",Buy,MRP and MPP planning,Transfer from TX2,...,11.0,11.0,11.0,11.0,49.0,823,823.0,True,51.0,772.0
1,NL1,00515-000,00515-000NL1,HUB RD 10 X 3.1 X 6 X 1.7 SS,FG-Active,FG Purchased Parts,"Saw, Miss Jia Wen",Buy,MRP and MPP planning,Transfer from TX2,...,NaN,NaN,NaN,NaN,NaN,0,0.0,True,0.0,0.0
2,NL1,01366-090,01366-090NL1,"CABLE ASSY,IEEE 1284,90 DEG",FG-Active,FG Purchased Parts,"Chong, Ms. Siau Yieng",Buy,MRP and MPP planning,Transfer from TX2,...,NaN,NaN,NaN,NaN,NaN,0,0.0,True,0.0,0.0
3,NL1,01473-000,01473-000NL1,FLAT SPRING ROLL HOLDER 2,FG-Active,FG Purchased Parts,"Chan, Ms. Yee Suen (Wendy)",Buy,MRP and MPP planning,Transfer from TX2,...,1.0,1.0,NaN,1.0,3.0,57,57.0,True,4.0,53.0
4,NL1,01600BK06045,01600BK06045NL1,"Wax Ribbon, 60mmx450m,1600; Economy Wax, 25mm ...",FG-Active,FG Purchased Ribbons,NaN,Buy,MRP and MPP planning,EMEA AMOR SA NANTES EUR,...,NaN,NaN,NaN,NaN,NaN,0,0.0,True,0.0,0.0


In [2]:
dfMRPinputforpivot1=dfMRP.drop(dfMRP.columns.difference(['ITEM','ITEM_PLAN','BeforeJune30','OH','AfterJune30','ORG',
                                                       'ITEM STATUS','FAMILY','DEFAULT BUYER','PO UNIT PRICE']), 1)

dfMRPinputforpivot1.head()

,ORG,ITEM,ITEM_PLAN,ITEM STATUS,DEFAULT BUYER,FAMILY,PO UNIT PRICE,OH,BeforeJune30,AfterJune30
0,NL1,00358-000,00358-000NL1,FG-Active,"Mak, Ms. Ee Tinh",SPS Parts,0.0,238,51.0,772.0
1,NL1,00515-000,00515-000NL1,FG-Active,"Saw, Miss Jia Wen",SPS Parts,0.0,5,0.0,0.0
2,NL1,01366-090,01366-090NL1,FG-Active,"Chong, Ms. Siau Yieng",SPS Parts,0.0,4,0.0,0.0
3,NL1,01473-000,01473-000NL1,FG-Active,"Chan, Ms. Yee Suen (Wendy)",SPS Parts,0.0,445,4.0,53.0
4,NL1,01600BK06045,01600BK06045NL1,FG-Active,NaN,Barcode Ribbons,0.0,648,0.0,0.0


In [3]:
ohpiv=dfMRPinputforpivot1.pivot_table(index='ITEM', columns='ORG', aggfunc='sum').fillna(0)
print(len(ohpiv))
ohpiv=ohpiv.drop(['PO UNIT PRICE'],axis=1)
ohpiv

23249


AfterJune30                                    BeforeJune30  \
ORG                      NL1    SG1  SH2   TX2  TX3    TX4  TX5          NL1   
ITEM                                                                           
00358-000              772.0    0.0  0.0   0.0  0.0  648.0  0.0         51.0   
00375-000                0.0    0.0  0.0   0.0  0.0    0.0  0.0          0.0   
00515-000                0.0    0.0  0.0   0.0  0.0    0.0  0.0          0.0   
01298                    0.0    0.0  0.0   0.0  0.0    0.0  0.0          0.0   
01366-090                0.0    0.0  0.0   0.0  0.0    0.0  0.0          0.0   
...                      ...    ...  ...   ...  ...    ...  ...          ...   
ZT62063-T2E0200Z        39.0    0.0  0.0   0.0  0.0    0.0  0.0         44.0   
ZT62063-T2G0100Z         0.0    7.0  0.0   0.0  0.0    0.0  0.0          0.0   
ZT62063-T2M01078         0.0    0.0  0.0  31.0  0.0  354.0  0.0          0.0   
ZT62063-T2P0100Z         0.0  227.0  0.0   0.0  0.0    0.0  0.0          0.0   
ZT62063-T2P0200Z         0.0    1.0  0.0   0.0  0.0    0.0  0.0          0.0   

                             ...                     OH                        \
ORG                SG1  SH2  ...  TX3   TX4  TX5    NL1  SG1  SH2    TX2  TX3   
ITEM                         ...                                                
00358-000          0.0  0.0  ...  0.0  11.0  0.0  238.0  0.0  0.0  899.0  0.0   
00375-000          0.0  0.0  ...  0.0   0.0  0.0    0.0  0.0  0.0  100.0  0.0   
00515-000          0.0  0.0  ...  0.0   0.0  0.0    5.0  0.0  0.0  575.0  0.0   
01298              0.0  0.0  ...  0.0   0.0  0.0    0.0  0.0  0.0  538.0  0.0   
01366-090          0.0  0.0  ...  0.0   0.0  0.0    4.0  0.0  0.0   52.0  0.0   
...                ...  ...  ...  ...   ...  ...    ...  ...  ...    ...  ...   
ZT62063-T2E0200Z   0.0  0.0  ...  0.0   0.0  0.0    1.0  0.0  0.0    0.0  0.0   
ZT62063-T2G0100Z   4.0  0.0  ...  0.0   0.0  0.0    0.0  0.0  0.0    0.0  0.0   
ZT62063-T2M01078   0.0  0.0  ...  0.0   0.0  0.0    0.0  0.0  0.0    0.0  0.0   
ZT62063-T2P0100Z  34.0  0.0  ...  0.0   0.0  0.0    0.0  0.0  0.0    0.0  0.0   
ZT62063-T2P0200Z   3.0  0.0  ...  0.0   0.0  0.0    0.0  3.0  0.0    0.0  0.0   

                            
ORG               TX4  TX5  
ITEM                        
00358-000         0.0  0.0  
00375-000         0.0  0.0  
00515-000         0.0  0.0  
01298             0.0  0.0  
01366-090         0.0  0.0  
...               ...  ...  
ZT62063-T2E0200Z  0.0  0.0  
ZT62063-T2G0100Z  0.0  0.0  
ZT62063-T2M01078  0.0  0.0  
ZT62063-T2P0100Z  0.0  0.0  
ZT62063-T2P0200Z  0.0  0.0  

[23249 rows x 21 columns]

In [4]:
#ITEM STATUS	ITEM FAMILY	DEFAULT BUYER	 PO Cost from MRP, there are duplicates since there are by org
dfMRPinputforpivot=dfMRPinputforpivot1.drop(dfMRPinputforpivot1.columns.difference(['ITEM','ITEM STATUS','FAMILY','DEFAULT BUYER','PO UNIT PRICE']), 1)
dfMRPinputforpivot=dfMRPinputforpivot.drop_duplicates(subset='ITEM',keep='first')
print(dfMRPinputforpivot.shape)
dfMRPinputforpivot.head()

(23249, 5)


,ITEM,ITEM STATUS,DEFAULT BUYER,FAMILY,PO UNIT PRICE
0,00358-000,FG-Active,"Mak, Ms. Ee Tinh",SPS Parts,0.0
1,00515-000,FG-Active,"Saw, Miss Jia Wen",SPS Parts,0.0
2,01366-090,FG-Active,"Chong, Ms. Siau Yieng",SPS Parts,0.0
3,01473-000,FG-Active,"Chan, Ms. Yee Suen (Wendy)",SPS Parts,0.0
4,01600BK06045,FG-Active,NaN,Barcode Ribbons,0.0


In [5]:
ohpiv= pd.merge(ohpiv,dfMRPinputforpivot, on='ITEM', how='left')
ohpiv

C:\Users\MJ7255\Anaconda3\lib\site-packages\pandas\core\reshape\merge.py:643: UserWarning: merging between different levels can give an unintended result (2 levels on the left,1 on the right)
  warnings.warn(msg, UserWarning)


,ITEM,"(AfterJune30, NL1)","(AfterJune30, SG1)","(AfterJune30, SH2)","(AfterJune30, TX2)","(AfterJune30, TX3)","(AfterJune30, TX4)","(AfterJune30, TX5)","(BeforeJune30, NL1)","(BeforeJune30, SG1)",...,"(OH, SG1)","(OH, SH2)","(OH, TX2)","(OH, TX3)","(OH, TX4)","(OH, TX5)",ITEM STATUS,DEFAULT BUYER,FAMILY,PO UNIT PRICE
0,00358-000,772.0,0.0,0.0,0.0,0.0,648.0,0.0,51.0,0.0,...,0.0,0.0,899.0,0.0,0.0,0.0,FG-Active,"Mak, Ms. Ee Tinh",SPS Parts,0.0000
1,00375-000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,100.0,0.0,0.0,0.0,FG-Active,"Long, Ms. Angeline Foong Hwang",SPS Parts,0.0914
2,00515-000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,575.0,0.0,0.0,0.0,FG-Active,"Saw, Miss Jia Wen",SPS Parts,0.0000
3,01298,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,538.0,0.0,0.0,0.0,FG-Active,"Castillo, Mayra Isabel (Mayra)",SPS Parts,0.0000
4,01366-090,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,52.0,0.0,0.0,0.0,FG-Active,"Chong, Ms. Siau Yieng",SPS Parts,0.0000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23244,ZT62063-T2E0200Z,39.0,0.0,0.0,0.0,0.0,0.0,0.0,44.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,FG-Active,"Lee, Eng Nee",ZT620,739.8800
23245,ZT62063-T2G0100Z,0.0,7.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,...,0.0,0.0,0.0,0.0,0.0,0.0,FG-Active,"Lee, Eng Nee",ZT620,0.0000
23246,ZT62063-T2M01078,0.0,0.0,0.0,31.0,0.0,354.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,FG-Active,"Lee, Eng Nee",ZT620,0.0000
23247,ZT62063-T2P0100Z,0.0,227.0,0.0,0.0,0.0,0.0,0.0,0.0,34.0,...,0.0,0.0,0.0,0.0,0.0,0.0,FG-Active,"Lee, Eng Nee",ZT620,739.7000


In [6]:
#arrange as excel
ohpiv.insert(1,'ITEM1',ohpiv['ITEM'])
mid = ohpiv['ITEM STATUS']
ohpiv.drop(labels=['ITEM STATUS'], axis=1, inplace = True)
ohpiv.insert(2, 'ITEM STATUS', mid)

In [7]:
mid = ohpiv['FAMILY']
ohpiv.drop(labels=['FAMILY'], axis=1, inplace = True)
ohpiv.insert(3, 'ITEM FAMILY', mid)

In [8]:
mid = ohpiv['DEFAULT BUYER']
ohpiv.drop(labels=['DEFAULT BUYER'], axis=1, inplace = True)
ohpiv.insert(4, 'DEFAULT BUYER', mid)

In [9]:
ohpiv.head()

,ITEM,ITEM1,ITEM STATUS,ITEM FAMILY,DEFAULT BUYER,"(AfterJune30, NL1)","(AfterJune30, SG1)","(AfterJune30, SH2)","(AfterJune30, TX2)","(AfterJune30, TX3)",...,"(BeforeJune30, TX4)","(BeforeJune30, TX5)","(OH, NL1)","(OH, SG1)","(OH, SH2)","(OH, TX2)","(OH, TX3)","(OH, TX4)","(OH, TX5)",PO UNIT PRICE
0,00358-000,00358-000,FG-Active,SPS Parts,"Mak, Ms. Ee Tinh",772.0,0.0,0.0,0.0,0.0,...,11.0,0.0,238.0,0.0,0.0,899.0,0.0,0.0,0.0,0.0000
1,00375-000,00375-000,FG-Active,SPS Parts,"Long, Ms. Angeline Foong Hwang",0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,100.0,0.0,0.0,0.0,0.0914
2,00515-000,00515-000,FG-Active,SPS Parts,"Saw, Miss Jia Wen",0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,5.0,0.0,0.0,575.0,0.0,0.0,0.0,0.0000
3,01298,01298,FG-Active,SPS Parts,"Castillo, Mayra Isabel (Mayra)",0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,538.0,0.0,0.0,0.0,0.0000
4,01366-090,01366-090,FG-Active,SPS Parts,"Chong, Ms. Siau Yieng",0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,4.0,0.0,0.0,52.0,0.0,0.0,0.0,0.0000


In [10]:
#ohpiv['AfterJune30']['gg']=''
ohpiv.loc[:, ('Net Reqs OH - Demand Before June 30', 'TX2')] = ohpiv['OH']['TX2']-ohpiv['BeforeJune30']['TX2']
ohpiv.loc[:, ('Available to Transfer', 'TX2')] = np.where(ohpiv['Net Reqs OH - Demand Before June 30']['TX2'] > 0,'Yes','No')
ohpiv.loc[:, ('OH - Demand Before June 30', 'NL1')] = ohpiv['OH']['NL1']-ohpiv['BeforeJune30']['NL1']
ohpiv.loc[:, ('OH - Demand Before June 30', 'SG1')] = ohpiv['OH']['SG1']-ohpiv['BeforeJune30']['SG1']
ohpiv.loc[:, ('OH - Demand Before June 30', 'SH2')] = ohpiv['OH']['SH2']-ohpiv['BeforeJune30']['SH2']
ohpiv.loc[:, ('OH - Demand Before June 30', 'TX3')] = ohpiv['OH']['TX3']-ohpiv['BeforeJune30']['TX3']
ohpiv.loc[:, ('Total Demand NL1', 'NL1')] = ohpiv['AfterJune30']['NL1']+ohpiv['BeforeJune30']['NL1']

KeyError: 'OH'

In [ ]:
ohpiv.loc[:, ('Net Reqs', 'NL1')] = ohpiv['OH']['NL1']-ohpiv['BeforeJune30']['NL1']

In [12]:
ohpiv.loc[:, ('Need Inventory', 'NL1')] = ""
ohpiv.loc[:, ('Calculation TX2- NL1', 'NL1')] = ""
ohpiv.loc[:, ('IR Qty (OH Avilable in TX2)', 'NL1')] = ""
ohpiv.loc[:, ('DP to Review - NL1', 'NL1')] = ""
ohpiv.loc[:, ('Sourcing', 'NL1')] = ""
ohpiv.loc[:, ('Transfer fm TX2?', 'NL1')] = ""

ohpiv.loc[:, ('GSP -NL1', 'NL1')] = ""
ohpiv.loc[:, ('Receipts', 'NL1')] = ""
#SG1
ohpiv.loc[:, ('Net Reqs', 'SG1')] = ohpiv['OH']['SG1']-ohpiv['BeforeJune30']['SG1']
ohpiv.loc[:, ('Need Inventory', 'SG1')] = np.where(ohpiv['Net Reqs']['SG1'] < 0,'Y','N')
ohpiv.loc[:, ('Calculation TX2 -( NL1 +SG1)', 'SG1')] = ""
ohpiv.loc[:, ('IR Qty (OH Avilable in TX2)', 'SG1')] = ""
ohpiv.loc[:, ('DP to Review - SG1', 'SG1')] = ""
ohpiv.loc[:, ('Sourcing', 'SG1')] = ""
ohpiv.loc[:, ('Transfer fm TX2?', 'SG1')] = ""
ohpiv.loc[:, ('Total Demand SG1', 'SG1')] = ""
ohpiv.loc[:, ('GSP -SG1', 'SG1')] = ""
ohpiv.loc[:, ('Receipts', 'SG1')] = ""
#SH2
ohpiv.loc[:, ('Net Reqs', 'SH2')] = ohpiv['OH']['SH2']-ohpiv['BeforeJune30']['SH2']
ohpiv.loc[:, ('Need Inventory', 'SH2')] = np.where(ohpiv['Net Reqs']['SH2'] < 0,'Y','N')
ohpiv.loc[:, ('Calculation TX2 -( NL1 +SG1+SH2)', 'SH2')] = ""
ohpiv.loc[:, ('IR Qty (OH Avilable in TX2)', 'SH2')] = ""
ohpiv.loc[:, ('DP to Review - SH2', 'SH2')] = ""
ohpiv.loc[:, ('Sourcing', 'SH2')] = ""
ohpiv.loc[:, ('Transfer fm TX2?', 'SH2')] = ""
ohpiv.loc[:, ('Total Demand SH2', 'SH2')] = ""
ohpiv.loc[:, ('GSP -SH2', 'SH2')] = ""
ohpiv.loc[:, ('Receipts', 'SH2')] = ""

In [13]:
ohpiv

AfterJune30                         BeforeJune30              \
ORG                      NL1    SG1  SH2    TX2  TX3          NL1    SG1  SH2   
ITEM                                                                            
00358-000              546.0    0.0  0.0  482.0  0.0        101.0    0.0  0.0   
00375-000                0.0    0.0  0.0    0.0  0.0          0.0    0.0  0.0   
00515-000                0.0    0.0  0.0    0.0  0.0          0.0    0.0  0.0   
01298                    0.0    0.0  0.0    0.0  0.0          0.0    0.0  0.0   
01366-090                0.0    0.0  0.0    0.0  0.0          0.0    0.0  0.0   
...                      ...    ...  ...    ...  ...          ...    ...  ...   
ZT62063-T2E0200Z        31.0    0.0  0.0    0.0  0.0         73.0    0.0  0.0   
ZT62063-T2G0100Z         0.0   10.0  0.0    0.0  0.0          0.0    5.0  0.0   
ZT62063-T2M01078         0.0    0.0  0.0  352.0  0.0          0.0    0.0  0.0   
ZT62063-T2P0100Z         0.0  120.0  0.0    0.0  0.0          0.0  107.0  0.0   
ZT62063-T2P0200Z         0.0    1.0  0.0    0.0  0.0          0.0    2.0  0.0   

                             ... Net Reqs Need Inventory  \
ORG                TX2  TX3  ...      SH2            SH2   
ITEM                         ...                           
00358-000         84.0  0.0  ...      0.0              N   
00375-000          0.0  0.0  ...      0.0              N   
00515-000          0.0  0.0  ...      0.0              N   
01298              0.0  0.0  ...      0.0              N   
01366-090          0.0  0.0  ...      0.0              N   
...                ...  ...  ...      ...            ...   
ZT62063-T2E0200Z   0.0  0.0  ...      0.0              N   
ZT62063-T2G0100Z   0.0  0.0  ...      0.0              N   
ZT62063-T2M01078  20.0  0.0  ...      0.0              N   
ZT62063-T2P0100Z   0.0  0.0  ...      0.0              N   
ZT62063-T2P0200Z   0.0  0.0  ...      0.0              N   

                 Calculation TX2 -( NL1 +SG1+SH2) IR Qty (OH Avilable in TX2)  \
ORG                                           SH2                         SH2   
ITEM                                                                            
00358-000                                                                       
00375-000                                                                       
00515-000                                                                       
01298                                                                           
01366-090                                                                       
...                                           ...                         ...   
ZT62063-T2E0200Z                                                                
ZT62063-T2G0100Z                                                                
ZT62063-T2M01078                                                                
ZT62063-T2P0100Z                                                                
ZT62063-T2P0200Z                                                                

                 DP to Review - SH2 Sourcing Transfer fm TX2?  \
ORG                             SH2      SH2              SH2   
ITEM                                                            
00358-000                                                       
00375-000                                                       
00515-000                                                       
01298                                                           
01366-090                                                       
...                             ...      ...              ...   
ZT62063-T2E0200Z                                                
ZT62063-T2G0100Z                                                
ZT62063-T2M01078                                                
ZT62063-T2P0100Z                                                
ZT62063-T2P0200Z                                             

In [11]:
strings = time.strftime("%m.%d.%Y")
strings= 'DC Movement ' + strings 
b=("%s.xlsx" % strings)

ohpiv.to_excel(b, sheet_name= 'Movement')

In [ ]:
oh piv by location
#ask for most recent template